In [ ]:
import pandas as pd
import os
import glob

# Menggunakan engine openpyxl untuk membaca file Excel dengan format aslinya
read_engine = "openpyxl"

# Mendapatkan daftar file yang sesuai di folder saat ini
file_list = sorted(glob.glob("All_Conversation_Halo_BCA_Chat*.xlsx"))

def extract_table(file):
    df_raw = pd.read_excel(file, header=None, engine=read_engine, keep_default_na=False, dtype=str)  

    # Mencari baris yang mengandung header (Kolom 1, Kolom 2, dst.)
    for i, row in df_raw.iterrows():
        if row.astype(str).str.contains("Download", case=False, na=False).any(): 
            start_row = i
            break
    else:
        raise ValueError(f"Header tidak ditemukan dalam file {file}")

    # Membaca ulang file dari baris yang mengandung header
    df = pd.read_excel(file, skiprows=start_row, engine=read_engine, keep_default_na=False, dtype=str)  

    # Menghapus baris yang memiliki sel dengan nilai tertentu beserta baris di bawahnya
    keywords = ["RESPONSE RATE", "PASSED RESPONSE RATE", "FAILED RESPONSE RATE", "ABANDONES RATE"] 
    rows_to_drop = df[df.apply(lambda row: row.astype(str).isin(keywords).any(), axis=1)].index
    for index in rows_to_drop:
        df = df.drop(index, errors='ignore')
        df = df.drop(index + 1, errors='ignore')  

    return df.reset_index(drop=True)

def extract_periode_date(file):
    df_raw = pd.read_excel(file, header=None, engine=read_engine, keep_default_na=False, dtype=str)  
    
    # Mencari baris yang mengandung kata "PERIODE"
    for i, row in df_raw.iterrows():
        for j, cell in enumerate(row):
            if isinstance(cell, str) and "PERIODE" in cell.upper():
                # Mengambil tanggal di sebelah kanan jika ada
                if j + 1 < len(row):
                    date_value = row[j + 1]
                    try:
                        return pd.to_datetime(date_value, errors='coerce').strftime("%d-%b-%Y")
                    except:
                        return None
    return None

# Menggabungkan semua tabel
dataframes = []
for file in file_list:
    df = extract_table(file)
    date = extract_periode_date(file)
    if date:
        df["Periode"] = date  # Format sudah dalam dd-mmm-yyyy
    
    dataframes.append(df)

merged_df = pd.concat(dataframes, ignore_index=True)

# Menghapus kolom yang berjudul "Unnamed"
merged_df = merged_df.loc[:, ~merged_df.columns.str.contains('^Unnamed')]

# Simpan hasil gabungan tanpa mengubah format asli
output_file = "Merged.xlsx"
merged_df.to_excel(output_file, index=False, engine=read_engine)
print(f"File berhasil digabungkan dan disimpan sebagai '{output_file}'")

File berhasil disimpan sebagai output.xlsx
